In [1]:
%run CPCAnalysisTool.py

In [2]:
import os, sys, re, requests, openpyxl, glob, math, time, random
import pandas as pd
import numpy as np

In [3]:
ArchivePath = r'C:\Users\Hong\Dropbox\Research\ProjectFolder\Archive'
USPTOFPagePath = r'C:\Users\Hong\AppData\Local\Programs\Python\Python36-32\19762012Pat\ASGNFinder'
USGVKeyDict = np.load(ArchivePath+r'\01_Misc\02_USGVKeyDict.npy').item()
USCompNameDict = np.load(ArchivePath+r'\01_Misc\02_USCompNameDict.npy').item()
USGVKeyDict['Alliance Semiconductor'] = 29323
USCompNameDict[29323] = 'Alliance Semiconductor'
USCompNameDict[116770] = USCompNameDict[116770].replace(r'/','')

SavingPath = r'.\03_110217(Cutoff15)'

# Lower Letter 
USLowerCaseFirstLetter = [x.lower().split(' ')[0] for x in list(USGVKeyDict)]
USLowerCaseException = []

for USComp in list(USGVKeyDict):
    CompNameSplit = USComp.lower().split(' ')
    if USLowerCaseFirstLetter.count(CompNameSplit[0]) == 1:
        try:
            USNameLowerDict[CompNameSplit[0]] = USComp
        except:
            USNameLowerDict = {CompNameSplit[0]:USComp}
    else:
        if CompNameSplit[0] not in USLowerCaseException:
            USLowerCaseException.append(CompNameSplit[0])
        try:
            USNameLowerDict[CompNameSplit[0]+' '+CompNameSplit[1]] = USComp
        except:
            USNameLowerDict = {CompNameSplit[0]+' '+CompNameSplit[1]:USComp}

In [4]:
CPCFilesPath = r"C:\Users\Hong\Dropbox\Research\2017 Final Data\03CPCAnalysis\100317\05_UltSubstUSPTO"
CPCFilesDict = {}

for files in glob.glob(CPCFilesPath+r'\*'):
    USGVKey = int(files.split(r'(')[1].split(r').xlsx')[0])
    try:
        CPCFilesDict[USGVKey] = files
    except:
        CPCFilesDict = {USGVKey:files}

ListofEquipCPCFiles = []
EqCPCFilesPath = r"C:\Users\Hong\Dropbox\Research\2017 Final Data\03CPCAnalysis\100317\05_UpstreamUSPTO"

for files in glob.glob(EqCPCFilesPath+r'\0*'):
    USGVKey = int(files.split(r'(')[1].split(r').xlsx')[0])
    ListofEquipCPCFiles.append(USGVKey)
    try:
        EqCPCFilesDict[USGVKey] = files
    except:
        EqCPCFilesDict = {USGVKey:files}

EqCPCFilesPath = r"C:\Users\Hong\Dropbox\Research\ProjectFolder\Archive\EquiSupplierUSPTO"
for files in glob.glob(EqCPCFilesPath+r'\0*'):
    USGVKey = int(files.split(r'(')[1].split(r').xlsx')[0])
    ListofEquipCPCFiles.append(USGVKey)
    EqCPCFilesDict[USGVKey] = files
    

In [5]:
# Keyword Search
OpenedFile = pd.read_excel(r'00_Company_List.xlsx')
OpenedFile = OpenedFile[OpenedFile['Type']==1]
CompNameList = list(OpenedFile['Semi Company'])
GVKeyList = list(OpenedFile['GVKey.1'])
Fabless = [14324,66708,24800,117768]
ExceptionList = [10499,185517]
SemiGVKeyList = [x for x in GVKeyList if x not in ExceptionList]

#CompNameList = list(OpenedFile[pd.notnull(OpenedFile['Equipment Company'])]['Equipment Company'])
#GVKeyList = list(OpenedFile[pd.notnull(OpenedFile['Equipment Company'])]['GVKey'])


In [6]:
# RDDCPC
CutoffValue = 15
CompanyCount = 1
AllSigSemiCPCList={}
AllSigSemiCPC = []
AllSigSemiCPCCount = []
OpenedFile = np.nan
for USGVKey in SemiGVKeyList:
    OpenedFile = pd.read_excel(CPCFilesDict[USGVKey])
    OpenedFile.dropna(subset=['CPC'], inplace=True)
    OpenedFile['FirstCPC'] = FirstCPCList(OpenedFile)
    OpenedFile['FirstSubclass'] = OpenedFile['FirstCPC'].apply(lambda x: x[:4])
    Candidate = list(set(OpenedFile['FirstSubclass']))
    AnalysisFile = RatioFinder(OpenedFile,Candidate)
    AnalysisFile['Cumul'] = AnalysisFile['Ratio'].cumsum()
    AllSigSemiCPCList = AnalysisFile[AnalysisFile['Ratio']>CutoffValue]
    AllSigSemiCPCCount = AllSigSemiCPCCount + list(AllSigSemiCPCList['CPC'])

RDDCPC = list(set([x for x in AllSigSemiCPCCount if AllSigSemiCPCCount.count(x)>=CompanyCount]))

In [7]:
# Keyword Search
OpenedFile = pd.read_excel(r'00_Company_List.xlsx')
#OpenedFile = OpenedFile[OpenedFile['Type']==1]
#CompNameList = list(OpenedFile['Semi Company'])
#GVKeyList = list(OpenedFile['GVKey.1'])

CompNameList = list(OpenedFile[pd.notnull(OpenedFile['Equipment Company'])]['Equipment Company'])
EquiGVKeyList = list(OpenedFile[pd.notnull(OpenedFile['Equipment Company'])]['GVKey'])


In [8]:
# RFPCPC
AllSigSemiCPCList={}
AllSigSemiCPC = []
AllSigSemiCPCCount = []
OpenedFile = np.nan
for USGVKey in EquiGVKeyList:
    OpenedFile = pd.read_excel(EqCPCFilesDict[USGVKey])
    OpenedFile.dropna(subset=['CPC'], inplace=True)
    OpenedFile['FirstCPC'] = FirstCPCList(OpenedFile)
    OpenedFile['FirstSubclass'] = OpenedFile['FirstCPC'].apply(lambda x: x[:4])
    Candidate = list(set(OpenedFile['FirstCPC']))
    AnalysisFile = RatioFinder(OpenedFile,Candidate)
    AnalysisFile['Cumul'] = AnalysisFile['Ratio'].cumsum()
    AllSigSemiCPCList = AnalysisFile[AnalysisFile['Ratio']>CutoffValue]
    AllSigSemiCPCCount = AllSigSemiCPCCount + list(AllSigSemiCPCList['CPC'])

RFPCPC = list(set([x for x in AllSigSemiCPCCount if AllSigSemiCPCCount.count(x)>=CompanyCount]))
print(RFPCPC)
RDDCPC = RDDCPC + RFPCPC

['G01N 21/', 'G01R 31/', 'B23K 20/', 'H01J 37/', 'G03F 7/', 'C23C 16/', 'B82Y 35/', 'H01L 21/', 'B82Y 10/']


In [9]:
# Cumulative
AllSigEquiCPCList=[]
for USGVKey in SemiGVKeyList:
    OpenedFile = pd.read_excel(CPCFilesDict[USGVKey])
    OpenedFile.dropna(subset=['CPC'], inplace=True)
    OpenedFile['FirstCPC'] = FirstCPCList(OpenedFile)
    Count, Ratio = CumuFinder(OpenedFile,RDDCPC)
    AllSigEquiCPCList.append({'Company Name':USCompNameDict[USGVKey]
                              ,'Count':Count, 'Ratio':Ratio})
AllSigEquiCPCList = pd.DataFrame(AllSigEquiCPCList)
AllSigEquiCPCList.sort_values(by=['Ratio'], ascending=[False], inplace=True)
writer = pd.ExcelWriter(SavingPath+r'\01_RDDCPCCumul(Cutoff'+str(CutoffValue)+r'CC'+str(CompanyCount)
                        +').xlsx', engine='xlsxwriter')
AllSigEquiCPCList.to_excel(writer, sheet_name='Sheet1')
writer.save()
AllSigEquiCPCList

,Company Name,Count,Ratio
0,ALTERA CORP,2292,92.643492
13,XILINX INC,2674,92.493947
1,ADVANCED MICRO DEVICES,8989,91.323783
9,MICRON TECHNOLOGY INC,18180,88.587857
11,NVIDIA CORP,1377,85.000000
8,LSI CORP,4107,83.492580
4,CYPRESS SEMICONDUCTOR CORP,1716,82.978723
7,INTL RECTIFIER CORP,910,82.502267
6,INTEL CORP,18168,82.446905
12,QUALCOMM INC,4694,76.586719


In [10]:
# Cumulative
AllSigEquiCPCList=[]
for USGVKey in EquiGVKeyList:
    OpenedFile = pd.read_excel(EqCPCFilesDict[USGVKey])
    OpenedFile.dropna(subset=['CPC'], inplace=True)
    OpenedFile['FirstCPC'] = FirstCPCList(OpenedFile)
    Count, Ratio = CumuFinder(OpenedFile,RFPCPC)
    AllSigEquiCPCList.append({'Company Name':USCompNameDict[USGVKey]
                              ,'Count':Count, 'Ratio':Ratio})
AllSigEquiCPCList = pd.DataFrame(AllSigEquiCPCList)
AllSigEquiCPCList.sort_values(by=['Ratio'], ascending=[False], inplace=True)
writer = pd.ExcelWriter(SavingPath+r'\01_RFPCPCCumul(Cutoff'+str(CutoffValue)+r'CC'+str(CompanyCount)
                        +').xlsx', engine='xlsxwriter')
AllSigEquiCPCList.to_excel(writer, sheet_name='Sheet1')
writer.save()
AllSigEquiCPCList

,Company Name,Count,Ratio
8,Varian Semiconductor Equipment,262,89.726027
0,ASML,1927,89.212963
5,Novellus,571,86.253776
13,ETEC Systems Inc,55,85.937500
2,ASM,784,84.573894
1,AppMat,4876,81.010135
4,LamResearch,1223,78.649518
3,KLA Tencor,797,76.560999
11,Kulicke,127,64.795918
12,Ultratech,46,63.888889


In [11]:
df1 = pd.concat([pd.DataFrame({'RDDCPC': RDDCPC}),pd.DataFrame({'RFPCPC': RFPCPC})], axis=1)
writer = pd.ExcelWriter(SavingPath+r'\01_CPCList(Cutoff'+str(CutoffValue)+r'CC'+str(CompanyCount)
                        +').xlsx', engine='xlsxwriter')
df1.to_excel(writer, sheet_name='Sheet1')
writer.save()


In [12]:
# R&D Specialization Rate From Here
CPCFilesPath = r"C:\Users\Hong\Dropbox\Research\2017 Final Data\03CPCAnalysis\100317\05_UltSubstUSPTO"
CPCAnalysisPath = SavingPath + r".\Analysis"

print(len(glob.glob(CPCFilesPath+r'\0*')))
for files in glob.glob(CPCFilesPath+r'\0*'):
    USGVKey = int(files.split(r'(')[1].split(r').xlsx')[0])
    if USGVKey not in SemiGVKeyList+ExceptionList:
        continue
    CompName = USCompNameDict[USGVKey]
    print(CompName)
    xl = pd.read_excel(files)
    xl = xl[pd.notnull(xl['CPC'])]

    all_dataTemp = []
    IntegratedCol = []
    xl['FirstCPC'] = FirstCPCList(xl)

    try:
        xl['FILED_USPTO'] = np.where(xl['FILED_USPTO']=='No File Date',xl['FILED'],xl['FILED_USPTO'])
        print(xl[xl['FILED_USPTO']=='No File Date'].index)
        xl.dropna(subset=['FILED_USPTO'], inplace = True) 
    except:
        pass

    xl['RDDCPC'] = np.nan
    xl['RFPCPC'] = np.nan

    for CPCItem in RDDCPC:
        xl['RDDCPC'] = np.where(xl['CPC'].str.contains(CPCItem),1,xl['RDDCPC'])
    for CPCItem in RFPCPC:
        xl['RFPCPC'] = np.where(xl['CPC'].str.contains(CPCItem),1,xl['RFPCPC'])

    MinYear = int(xl['FILED_USPTO'].min())
    MaxYear = int(xl['FILED_USPTO'].max())
    
    for k in range(MinYear, MaxYear+1):
        xlTemp = xl[xl['FILED_USPTO'] == k]
        TotalPat = xlTemp['FILED_USPTO'].count()
        RDDCPCCount = xlTemp['RDDCPC'].count()
        RFPCPCCount = xlTemp['RFPCPC'].count()
        if RDDCPCCount!= 0:
            RnDSpeR = (RDDCPCCount-RFPCPCCount)/RDDCPCCount*100
        else:
            RnDSpeR =np.nan
        all_dataTemp.append({'Year':k, 'GVKey': USGVKey, 'CompName':CompName, 'TotalPat':TotalPat, 
                             'SemiPat':RDDCPCCount,'EquiPat':RFPCPCCount, 'RnDSpeR':RnDSpeR})

    df = pd.DataFrame(all_dataTemp)
#    df2 = df[df['TotalPat']>=10]
    ax = df.plot(x = "Year", y = "RnDSpeR", ylim = [0,100])
    fig = ax.get_figure()
    fig.savefig(CPCAnalysisPath+r'\02_'+CompName+r'('+str(USGVKey)+r')'+'Graph.png')
    df.sort_values(by=['GVKey', 'Year'], ascending=[True, True], inplace=True)

    try:
        RnDSpecFile = pd.concat([RnDSpecFile,df])
    except:
        RnDSpecFile = df
    writer = pd.ExcelWriter(CPCAnalysisPath+r'\06_'+CompName+r'('+str(USGVKey)+r')'+'.xlsx', engine='xlsxwriter')
    df.to_excel(writer, sheet_name='Sheet1')
    writer.save()

writer = pd.ExcelWriter(CPCAnalysisPath+r'\06_CPCAnalysis.xlsx', engine='xlsxwriter')
RnDSpecFile.to_excel(writer, sheet_name='Sheet1')
writer.save()


57
ADVANCED MICRO DEVICES
Int64Index([], dtype='int64')
ALTERA CORP
Int64Index([], dtype='int64')
ANALOG DEVICES
Int64Index([], dtype='int64')
BROADCOM CORP
Int64Index([], dtype='int64')
CYPRESS SEMICONDUCTOR CORP
Int64Index([], dtype='int64')
DALLAS SEMICONDUCTOR CORP


c:\users\hong\appdata\local\programs\python\python36-32\lib\site-packages\pandas\core\ops.py:792: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = getattr(x, name)(y)


FREESCALE SEMICONDUCTOR LTD
Int64Index([], dtype='int64')
INTEL CORP
Int64Index([], dtype='int64')
INTL RECTIFIER CORP
Int64Index([], dtype='int64')
LSI CORP
MICRON TECHNOLOGY INC
Int64Index([], dtype='int64')
NATIONAL SEMICONDUCTOR CORP
NVIDIA CORP
Int64Index([], dtype='int64')
QUALCOMM INC
Int64Index([], dtype='int64')
TEXAS INSTRUMENTS INC
Int64Index([], dtype='int64')
XILINX INC
ZILOG INC


In [13]:
# Equipment Files
UselessColList = ['Data Date', 'Industry Format', 'Level of Consolidation - Company Annual Descriptor', 'Population Source', 'Data Format']
UselessColListNU = []

NAColumnNames = {'Global Company Key':'GVKey', 'Data Year - Fiscal':'Year', 'ISO Currency Code':'CurrencyCode',
                 'Current Assets - Total':'Current Assets','Assets - Total':'Assets', 'Capital Expenditures':'CapEx', 
                 'Revenue - Total':'Revenue','Research and Development Expense':'RnDEx',
                 'Active/Inactive Status Marker':'A/I','Current ISO Country Code - Headquarters':'CountryCode',
                 'North American Industry Classification Code':'NAICS','Standard Industry Classification Code':'SICS'}
FinancialData = np.nan

for CompustatFile in glob.glob(ArchivePath+r'\01_Compustat\01_33*.xlsx'):
    Openedfile = pd.read_excel(CompustatFile)
    try:
        FinancialData = pd.concat([FinancialData,Openedfile])
    except:
        FinancialData = Openedfile
for UselessColumns in UselessColList:
    try:
        FinancialData.drop(UselessColumns, axis=1, inplace=True)
    except:
        pass

FinancialData.rename(columns = NAColumnNames, inplace = True)

YearstoRemove = [x for x in list(FinancialData[FinancialData['GVKey']==157858]['Year']) 
                 if x in list(FinancialData[FinancialData['GVKey']==185517]['Year'])]
for YeartoRemove in YearstoRemove:
    FinancialData = FinancialData.drop(FinancialData[(FinancialData['GVKey'] == 157858) & (FinancialData['Year'] == YeartoRemove)].index)

FreescaleCompName = list(FinancialData[FinancialData['GVKey']==185517]['Company Name'])[0]
FinancialData['Company Name'] = np.where(FinancialData['GVKey']==157858, FreescaleCompName, FinancialData['Company Name'])
FinancialData['GVKey'] = np.where(FinancialData['GVKey']==157858, 185517,FinancialData['GVKey'])

FinancialData = FinancialData[FinancialData["Year"]<2013]

#Age
FinancialData['MinYear'] = np.nan

for USGVKey in list(FinancialData['GVKey']):
    FinancialData['MinYear'] = np.where(FinancialData['GVKey']==USGVKey, 
                                        min(FinancialData[FinancialData['GVKey']==USGVKey]['Year']),FinancialData['MinYear'])
FinancialData['Age'] = FinancialData['Year'] - FinancialData['MinYear']+1


In [14]:
ResultFile = np.nan
for USGVKey in SemiGVKeyList+ExceptionList:
    filename = [x for x in glob.glob(SavingPath+r'\Analysis\*') 
                if '('+str(USGVKey)+').xlsx' in x][0]
    CompName = USCompNameDict[USGVKey]
    OpenedFile = pd.read_excel(filename)
    OpenedFile = pd.merge(OpenedFile, FinancialData[FinancialData['GVKey']==USGVKey], on = 'Year', how='left')
    try:
        ResultFile = pd.concat([ResultFile,OpenedFile])
    except:
        ResultFile = OpenedFile

ResultFile = ResultFile[ResultFile['Year']<2013]
ResultFile = ResultFile[ResultFile['Year']>1958]
# Apply 
Openedfile = pd.read_excel(r'C:\Users\Hong\Dropbox\Research\ProjectFolder\Results\101217\02_EquiIndustryResult.xlsx')
values_dict = {Years:list(Openedfile[Openedfile['Year']==Years]['IndustryRnDEx'])[0] for Years in list(Openedfile['Year'])}
ResultFile['IndustryRnDEx'] = ResultFile['Year'].map(values_dict)

writer = pd.ExcelWriter(SavingPath+r'\07_CPC+EquipFinancials.xlsx', engine='xlsxwriter')
ResultFile.to_excel(writer, sheet_name='Sheet1')
writer.save()
